# Advent of Code - 2024 - Day 20 - Problem 1

https://adventofcode.com/2024/day/20

## Load Source Data

Load the map data into `DATA`.

In [5]:
f = open("data/day20.txt", "r")
DATA = list(map(str.strip, f.readlines()))
f.close()

DATA="""###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############"""
DATA = list(map(str.strip, DATA.splitlines()))

DATA

['###############',
 '#...#...#.....#',
 '#.#.#.#.#.###.#',
 '#S#...#.#.#...#',
 '#######.#.#.###',
 '#######.#.#...#',
 '#######.#.###.#',
 '###..E#...#...#',
 '###.#######.###',
 '#...###...#...#',
 '#.#####.#.###.#',
 '#.#...#.#.#...#',
 '#.#.#.#.#.#.###',
 '#...#...#...###',
 '###############']

## Create Map Class

In [8]:
class Map:

    def __init__(self, lines):
        self._lines = list(map(list, DATA))
        self._rows = len(self._lines)
        self._cols = len(self._lines[0])

        # Find the starting position
        #
        for row in range(self._rows):
            for col in range(self._cols):
                if self._lines[row][col] == "S":
                    self._start = (row, col)
                    break

## Create Path Class

In [9]:
class Path:

    def __init__(self, position):
        self._position = position

## Find Solutions

In [4]:
MAP = Map(DATA)


304
